In [ ]:
import ee
import os
from tqdm import tqdm
import calendar
import json
from google.cloud import storage
import time


default_s2_bands = ['B2', 'B3', 'B4', 'B8',  # 10m
                    'B5', 'B8A', 'B11']  # 20m


def combine_pair(i):
    i = ee.Number(i)
    i10 = ee.Image(l10.get(i))
    i20 = ee.Image(l20.get(i))
    return (ee.Image.cat([
                i10.select(['B2', 'B3', 'B4', 'B8', 'cloudAndCloudShadowMask']),
                i20.select(['B5', 'B8A', 'B11'])
        ]).copyProperties(i10, ['system:time_start', 'system:index']))


def upsample20mBands(img):
    ref10 = img.select('B2').projection()

    bands20_to10 = (img.select(['B5', 'B8A', 'B11'])
                    .resample('bilinear')
                    .reproject(ref10))
    bands10 = img.select(['B2', 'B3', 'B4', 'B8'])
    mask10 = img.select('cloudAndCloudShadowMask')
    out = (ee.Image.cat([bands10, bands20_to10, mask10])
            .updateMask(mask10.eq(0))
            .copyProperties(img, img.propertyNames()))
    return out



PROJECT_NAME = 'treeai-470815'
EXP_SCALE = 10
MAX_PIXELS = 1e13

# Initialize the Earth Engine module and register this code in project 'TreeAI'
ee.Authenticate()
ee.Initialize(project=PROJECT_NAME)

swiss_boundary = ee.FeatureCollection("projects/treeai-470815/assets/swissBOUNDARIES3D_1_5_TLM_KANTONSGEBIET")

start_date = ee.Date.fromYMD(2018, 1, 1)
end_date = start_date.advance(1, 'month')

col = (ee.ImageCollection("projects/satromo-prod/assets/col/S2_SR_HARMONIZED_SWISS")
        .filterBounds(swiss_boundary.geometry())
        .filterDate(start_date, end_date))

images_10m = col.filter(ee.Filter.eq('pixel_size_meter', 10)).sort('system:time_start', True)
images_20m = col.filter(ee.Filter.eq('pixel_size_meter', 20)).sort('system:time_start', True)

n = images_10m.size()
if n.getInfo()>0:
    l10 = images_10m.toList(n)
    l20 = images_20m.toList(n)

    merged = ee.ImageCollection(ee.List.sequence(0, n.subtract(1)).map(combine_pair))
    merged = merged.map(upsample20mBands)
    monthly_median = merged.select(default_s2_bands).median().clip(swiss_boundary.geometry())

proj = monthly_median.select('B2').projection()
print("Projection object:", proj.getInfo())

crs = proj.crs().getInfo()
print("CRS:", crs)

info = monthly_median.getInfo()
print("Type:", info['type'])
print("Bands:", [b['id'] for b in info['bands']])

# task = ee.batch.Export.image.toCloudStorage(
#     image=monthly_median,
#     description="swiss_monthly_201801",
#     bucket="treeai_swiss",
#     fileNamePrefix="Sentinel2/2018_01",
#     region=swiss_boundary.geometry(),
#     scale=10,
#     crs="EPSG:2056",
#     maxPixels=1e13
# )
# task.start()

Projection object: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
CRS: EPSG:4326
Type: Image
Bands: ['B2', 'B3', 'B4', 'B8', 'B5', 'B8A', 'B11']


In [3]:
import geemap

In [ ]:
vis_params = {
    "bands": ["B4", "B3", "B2"], 
    "min": 0, 
    "max": 3000
}

In [4]:
Map = geemap.Map()
Map.centerObject(swiss_boundary, 7)

monthly_vis = monthly_median.reproject(crs="EPSG:32632", scale=100)
Map.addLayer(monthly_vis, vis_params, "Monthly median (preview)")
Map

Map(center=[46.80316652470189, 8.227942569584531], controls=(WidgetControl(options=['position', 'transparent_b…